In [101]:
libraries = c("dplyr", "tidyverse", "ggpubr")
for(x in libraries) {library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE)}

theme_set(theme_bw())

In [102]:
read.csv("../data/df_inci_final.csv") -> df_inci
read.csv("../data/SAR_caseprop_Reff.csv") -> df_Reff
read.csv("../data/flight/flight_matrix.csv") -> flight_matrix

In [103]:
## Reff_i & G_i
df_inci %>% mutate(Infections = df_Reff$Infections[which.min(abs(cum_icni_prop-df_Reff$Infections))]) -> temp
merge(temp, df_Reff %>% dplyr::select(Infections, Reff_1), by=c("Infections"), all.x=TRUE) %>% 
dplyr::select(-Infections) %>% rename(Reff_i=Reff_1) %>% mutate(G_i=Reff_i*MA_new_cases) %>%
mutate(location=case_when(location==c("Democratic Republic of Congo")~c("Congo, Democratic Republic of the"),
                          location==c("Curacao")~c("Curaçao"),
                          location==c("Czechia")~c("Czech Republic"),
                          location==c("Iran")~c("Iran, Islamic Republic of"),
                          location==c("South Korea")~c("Korea, Republic of"),
                          location==c("United States")~c("United States of America"),
                          location==c("Venezuela")~c("Venezuela, Bolivarian Republic of"),
                          location==c("Moldova")~c("Moldova, Republic of"),
                          location==c("Russia")~c("Russian Federation"),
                          TRUE~location)) %>%
filter(!(location %in% c("Gibraltar", "Ghana", "Liberia", "Sudan", 
                         "Guadeloupe", "Greenland", "Saint Martin (French part)"))) -> input

as.Date(input$date) -> input$date
input %>% mutate(time=as.numeric(date-time_0)) -> input

Warning message in cum_icni_prop - df_Reff$Infections:
“longer object length is not a multiple of shorter object length”


In [104]:
is.element(unique(input$location), flight_matrix$destination) -> temp; which(temp==c("FALSE"))

integer(0)

In [105]:
## F_i
path <- "../data/flight/all_region/"; list.files(path = path, pattern = "*xlsx") -> file_list
substr(file_list,1,nchar(file_list)-5) -> flight_list
unique(input$location) -> country_list
sort(flight_list) -> flight_list_sort; sort(country_list) -> country_list_sort

## coverting NAs to zero in the flight data
flight_matrix[is.na(flight_matrix)] <- 0

# colnames(flight_matrix) -> list_temp; list_temp[4:length(list_temp)] -> country_list

In [106]:
w<-14 ## window time 
time_0 <- as.Date("2022-04-30")

i<-3 # temp

input %>% filter(location==country_list_sort[i]) %>% dplyr::select(pop2022) %>% distinct() %>% as.numeric() -> N_j
flight_matrix %>% filter(destination==country_list_sort[i])

# w/365*V_ij/N_j

X,destination,series,Afghanistan,Albania,Algeria,American.Samoa,Andorra,Angola,Anguilla,⋯,United.Kingdom,United.States.of.America,Uruguay,Uzbekistan,Vanuatu,Venezuela..Bolivarian.Republic.of,Viet.Nam,Yemen,Zambia,Zimbabwe
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,Aruba,TFR,0,0,0,0,0,0,0,⋯,10882,838288,2049,0,0,14958,0,0,0,0


In [121]:
country_list_sort

[1] "Andorra"                           "Argentina"                        
 [3] "Aruba"                             "Australia"                        
 [5] "Austria"                           "Bahamas"                          
 [7] "Bahrain"                           "Barbados"                         
 [9] "Belgium"                           "Benin"                            
[11] "Bermuda"                           "Bolivia"                          
[13] "Bosnia and Herzegovina"            "Brazil"                           
[15] "Bulgaria"                          "Cameroon"                         
[17] "Canada"                            "Central African Republic"         
[19] "Chile"                             "China"                            
[21] "Colombia"                          "Congo"                            
[23] "Congo, Democratic Republic of the" "Costa Rica"                       
[25] "Croatia"                           "Cuba"                             
[27] "Curaçao"                           "Cyprus"                           
[29] "Czech Republic"                    "Denmark"                          
[31] "Dominican Republic"                "Ecuador"                          
[33] "Egypt"                             "El Salvador"                      
[35] "Estonia"                           "Finland"                          
[37] "France"                            "Georgia"                          
[39] "Germany"                           "Greece"                           
[41] "Guam"                              "Guatemala"                        
[43] "Guyana"                            "Honduras"                         
[45] "Hungary"                           "Iceland"                          
[47] "India"                             "Indonesia"                        
[49] "Iran, Islamic Republic of"         "Ireland"                          
[51] "Israel"                            "Italy"                            
[53] "Jamaica"                           "Japan"                            
[55] "Jordan"                            "Korea, Republic of"               
[57] "Latvia"                            "Lebanon"                          
[59] "Lithuania"                         "Luxembourg"                       
[61] "Malta"                             "Martinique"                       
[63] "Mexico"                            "Moldova, Republic of"             
[65] "Monaco"                            "Montenegro"                       
[67] "Morocco"                           "Netherlands"                      
[69] "New Caledonia"                     "New Zealand"                      
[71] "Nigeria"                           "Norway"                           
[73] "Panama"                            "Paraguay"                         
[75] "Peru"                              "Philippines"                      
[77] "Poland"                            "Portugal"                         
[79] "Puerto Rico"                       "Qatar"                            
[81] "Romania"                           "Russian Federation"               
[83] "Saudi Arabia"                      "Serbia"                           
[85] "Singapore"                         "Slovakia"                         
[87] "Slovenia"                          "South Africa"                     
[89] "Spain"                             "Sweden"                           
[91] "Switzerland"                       "Thailand"                         
[93] "Turkey"                            "Ukraine"                          
[95] "United Arab Emirates"              "United Kingdom"                   
[97] "United States of America"          "Uruguay"                          
[99] "Venezuela, Bolivarian Republic of"

In [129]:
F_i_country <- list(); F_i_time_list <- list()

for(i in 1:length(country_list_sort)){
    flight_matrix %>% filter(destination==country_list_sort[i]) %>% t() -> temp
    temp[4:nrow(temp),] %>% as.matrix() -> temp

    flight_list %>% as.matrix() -> flight_list_matrix
    cbind(temp, flight_list_matrix) %>% as.data.frame() %>% rename(value=V1, location=V2) %>%
    dplyr::select(location, value) -> V_ij_temp; rownames(V_ij_temp) <- NULL

    input %>% dplyr::select(location, pop2022) %>% distinct() -> N_j_temp

    merge(V_ij_temp, N_j_temp, by=c("location"), all.x=TRUE) %>% na.omit() -> temp

    for(g in 1:max(input$time)) {
        input %>% filter(time==g) %>% dplyr::select(location, G_i) -> G_i_time
        merge(temp, G_i_time, by=c("location"), all.x=TRUE) -> temp_G_i_time
        temp_G_i_time$G_i[is.na(temp_G_i_time$G_i)] <- 0
        temp_G_i_time$value <- as.numeric(temp_G_i_time$value)
        temp_G_i_time %>% mutate(F_i_t = w/365*value/pop2022*G_i, time=g) -> temp_G_i_time 
        sum(temp_G_i_time$F_i_t) -> F_i_time_list[[g]]
    }
    
    do.call("rbind", F_i_time_list) %>% as.data.frame() %>% 
    mutate(time=1:max(input$time), location=country_list_sort[i]) %>% rename(F_i=V1) -> F_i_country[[i]]
}

In [131]:
do.call("rbind", F_i_country) %>% as.data.frame() -> F_i_country_all
merge(input, F_i_country_all, by=c("location", "time"), all.x=TRUE) -> input_final

In [132]:
input_final %>% head()

write.csv(input_final, "..")

location,time,X,iso_code,region,sub_region,date,new_cases,total_cases,MA_new_cases,MA_total_cases,date_import,MSM_pop,pop2022,cum_icni_prop,Reff_i,G_i,F_i
<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<date>,<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Andorra,100,4269,AND,Europe,Southern Europe,2022-08-08,0,4,0.14285714,3.285714,2022/07/25,576.5777,79.824,0.005698650,1.692379,0.2417684,413.8603
Andorra,101,4351,AND,Europe,Southern Europe,2022-08-09,0,4,0.07142857,3.357143,2022/07/25,576.5777,79.824,0.005822533,1.692379,0.1208842,365.6487
Andorra,102,4434,AND,Europe,Southern Europe,2022-08-10,0,4,0.07142857,3.428571,2022/07/25,576.5777,79.824,0.005946417,1.692379,0.1208842,335.2218
Andorra,103,4518,AND,Europe,Southern Europe,2022-08-11,0,4,0.07142857,3.500000,2022/07/25,576.5777,79.824,0.006070301,1.692379,0.1208842,355.1288
Andorra,104,4601,AND,Europe,Southern Europe,2022-08-12,0,4,0.07142857,3.571429,2022/07/25,576.5777,79.824,0.006194184,1.692379,0.1208842,355.1288
Andorra,105,4685,AND,Europe,Southern Europe,2022-08-13,0,4,0.07142857,3.642857,2022/07/25,576.5777,79.824,0.006318068,1.692379,0.1208842,355.1288
Andorra,106,4769,AND,Europe,Southern Europe,2022-08-14,0,4,0.07142857,3.714286,2022/07/25,576.5777,79.824,0.006441952,1.692379,0.1208842,453.8692
Andorra,107,4854,AND,Europe,Southern Europe,2022-08-15,0,4,0.07142857,3.785714,2022/07/25,576.5777,79.824,0.006565835,1.692379,0.1208842,329.7271
Andorra,108,4939,AND,Europe,Southern Europe,2022-08-16,0,4,0.07142857,3.857143,2022/07/25,576.5777,79.824,0.006689719,1.692379,0.1208842,266.4640


In [ ]:
merge(input, F_i_country, by=c("location", "time"), all.x=TRUE) -> input_final

In [2]:
r=0.01
(1+1.3*r*3.5)/(1+r*3.5)

r=0.05
(1+1.3*r*3.5)/(1+r*3.5)

r=0.07
(1+1.3*r*3.5)/(1+r*3.5)

[1] 1.010145

[1] 1.044681

[1] 1.059036